This notebook generates a JSON file that is later used by the front end.
It fetches every emoji, create an entry for each one and then save it in a JSON file.


# Initialisation

In [1]:
# Remove any existing content
!rm -rf repo
# We clone the repo in the repo folder
!git clone https://github.com/microsoft/fluentui-emoji.git repo
# Delete the .git folder because we don't need it
!rm -rf repo/.git

Cloning into 'repo'...
remote: Enumerating objects: 27960, done.
remote: Counting objects: 100% (367/367), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 27960 (delta 42), reused 352 (delta 33), pack-reused 27593
Receiving objects: 100% (27960/27960), 125.19 MiB | 840.00 KiB/s, done.
Resolving deltas: 100% (3291/3291), done.
Updating files: 100% (12041/12041), done.


# Transform the repo into a dictionary

In [8]:

import json
import os

"""This function will find all the emoji in a given folder
"""
def getAllImageOfFolder(path:str):
    # Get the path without "repo"
    pathForCDN = path[5:]+"/"

    
    result = {}
    # Try to get the 3D icon
    if os.path.exists(path+"/3D"):
        result["3D"] = pathForCDN+"3D/"+os.listdir(path+"/3D")[0] #os.listdir(path+"/3D")[0] get the first file in the folder
    # Try to get the color icon
    if os.path.exists(path+"/Color"):
        result["Color"] = pathForCDN+ "Color/"+os.listdir(path+"/Color")[0]

    # Try to get the Flat icon
    if os.path.exists(path+"/Flat"):
        result["Flat"] = pathForCDN+ "Flat/"+os.listdir(path+"/Flat")[0]
    if os.path.exists(path+"/High Contrast"):
        result["High Contrast"] = pathForCDN+"High Contrast/"+os.listdir(path+"/High Contrast")[0]
    return result

result = {}

        
# Iterate over the folders in the repo
for folder in os.listdir('repo/assets/'):
    # On macos, we have a .DS_Store file
    if folder != ".DS_Store" and os.path.isdir("repo/assets/"+folder):
        data = {
            "name":folder,
            "variant":[],

        }
        # Read the metadata json file
        with open('repo/assets/'+folder+'/metadata.json') as json_file:
            metadata = json.load(json_file)
            # Get the emoji in unicode : 🙂
            data["glyph"] = metadata["glyph"]
            # Get the keywords
            data["keywords"] = metadata["keywords"]


        # We check if the emoji has variants. If 3D is present, it means that the emoji has no variants
        if os.path.isdir('repo/assets/' + folder+ "/3D"):
            # Case where the emoji has no variants
            data["variant"].append(getAllImageOfFolder('repo/assets/' + folder))

        else:
            # Case where the emoji has variants
            for variant in os.listdir('repo/assets/' + folder):
                # We check if the variant is a folder and not a file
                if os.path.isdir('repo/assets/' + folder+ "/"+variant):
                    data["variant"].append(getAllImageOfFolder('repo/assets/' + folder+"/"+variant))

        result[folder] = data


# Write the result in a json file
with open('data.js', 'w') as outfile:
    toWrite= "/* eslint-disable import/no-anonymous-default-export */ \n export default "+ json.dumps(result, indent=4) 
    outfile.write(toWrite)
    print ("File written")

# Close the file
outfile.close()


File written
